In [1]:
import numpy as np
#Pan Sharpening Demo
import skimage
import skimage.io
import skimage.transform

In [2]:
#Channels, 2 blue, 3 green, 4 red
def sharpen(root):
    path="/Users/yamaoka/landsat/downloads/"+root+"/clipped/"

    r=skimage.io.imread(path+root+"_B4.TIF", as_grey=True) #Load Red
    g=skimage.io.imread(path+root+"_B3.TIF", as_grey=True) #Load Green
    b=skimage.io.imread(path+root+"_B2.TIF", as_grey=True) #Load Blue
    rgb= np.dstack((r,g,b )) #Combine into correctly ordered stack

    landsat_highres=path+root+"_B8.TIF"
    pan=skimage.io.imread(landsat_highres, as_grey=True)/65535 #Load chromatic, normalize to between 0 and 1

    rgb_big=skimage.transform.resize(rgb, output_shape=(pan.shape[0],pan.shape[1],3), order=3, mode='constant', cval=0.0) #Resize the rgb composite to match the chromatic
    hsv = skimage.color.rgb2hsv(rgb_big) #Conver the rgb to the HSV colorspace
    hsv[...,2]=pan #Poor man's pan sharpening, replace the intensity channel with the observed chromatic image
    rgb_pan = skimage.color.hsv2rgb(hsv) #Convert back to rgb space
    return rgb_pan


In [3]:
landsat1=sharpen(root="LC80270392017030LGN00") #File name of your imagery

In [5]:
from scipy import ndimage as ndi
from math import floor

landsat1_dog = landsat1 - ndi.gaussian_filter(landsat1, 15) #Difference of gaussians

In [6]:
stack=np.dstack((landsat1, landsat1_dog))
depth=stack.shape[2]
edge=3
buff=floor(edge/2)
stack_window=skimage.util.view_as_windows(np.pad(stack,pad_width=((buff,buff), (buff,buff),(0,0)), mode ='minimum'), window_shape=(edge,edge,depth), step=1) 
stack_window.shape #3 by 3 moving window around each pixel

stack_flat=stack_window.reshape(-1,edge*edge*depth) ; stack_flat.shape #Flatten to rows

(7639767, 54)

In [42]:
#make labels
building_small= skimage.io.imread('/Users/yamaoka/landuse/labels/texas-latest-free.shp/austin_tx_building.tif', as_grey=True) #Load Buildings
landuse_small = skimage.io.imread('/Users/yamaoka/landuse/labels/texas-latest-free.shp/austin_tx_landuse.tif', as_grey=True) #Load Landuse tmp

building= skimage.transform.resize(building_small, output_shape=(stack.shape[0],stack.shape[1]), order=3, mode='constant', cval=1.0) #Resize the labels
landuse = skimage.transform.resize(landuse_small, output_shape=(stack.shape[0],stack.shape[1]), order=3, mode='constant', cval=1.0) #Resize the labels

In [89]:
ll = landuse.flatten()
lb = building.flatten()

In [90]:
ll[ll == 1.0] = 0
lb[lb == 1.0] = 0
ll[ll != 0.0] = 1
lb[lb != 0.0] = 2
labels = ll+lb
labels[labels==3] = 2 #i know this is stupid need to fix later

[ 1.  1.  1. ...,  1.  1.  1.]
[ 0.99995532  0.99998395  0.99900523 ...,  0.99982725  0.99983609
  0.99999119]
[ 0.  0.  0. ...,  0.  0.  0.]


In [91]:
x_test=stack_flat
y_test=labels

x_train=x_test[y_test>0,...]
y_train=y_test[y_test>0]

y_train = y_train.astype(int)

proportions=1/(np.bincount(y_train)/y_train.shape)
weights = proportions/sum(proportions[1:])
weights = proportions[y_train]

[     0 769661 150222]
[ 1.19517944  1.19517944  1.19517944 ...,  1.19517944  1.19517944
  1.19517944]


/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [94]:
import sklearn.ensemble
rf=sklearn.ensemble.RandomForestClassifier(n_estimators=100, criterion='gini',
max_depth=None, min_samples_split=2, min_samples_leaf=1,
max_features='auto', bootstrap=True,
oob_score=True, n_jobs=2, random_state=0, verbose=True)

rf.fit(x_train,y_train, sample_weight=weights)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 39.3min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 82.3min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=2, oob_score=True, random_state=0,
            verbose=True, warm_start=False)

In [97]:
rf.oob_score_
import pickle
filename='rf100_first.sav'
pickle.dump(rf, open(filename, 'wb'))

import pickle


In [98]:
labelsfixed = labels.reshape(landuse.shape)
predictions=rf.predict(x_test)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  5.9min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 15.6min finished


NameError: name 'labels_predicted_andknown' is not defined

In [100]:
labels_predicted=predictions.reshape(labelsfixed.shape)
#labels_predicted_andknown[labelsfixed>0]=labelsfixed[labelsfixed>0]
#inalcolors=np.array([[0,0,0], #0 unlabeled
#                     [32,66,239], #1 water
#                     [0,102,0], #2 vegitation
#                     [102,51,0], #3 buildings
#                     [0,0,0], #4 rail
#                     [128,128,128]])#5 road
#outpath = "out/path/here"
#skimage.io.imsave("rf_predictions.png", labels_predicted)
#skimage.io.imsave("rf_predictions_andknown.png", finalcolors[labels_predicted_andknown])


/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/skimage/io/_io.py:132: UserWarning: rf_predictions.png is a low contrast image
  warn('%s is a low contrast image' % fname)


In [109]:
testcolors=np.array([0,0.8,0.2]) 
skimage.io.imsave("rf_predictions.png", testcolors[labels_predicted],)
skimage.io.imsave("dog.png", landsat1_dog)

/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
